In [1]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

USER = os.environ["USER"]

# NORMALIZATION WILL BE ACROSS ELECTRODE

In [2]:
src = "/home/andres/repos/rat-mapping-paper/reports/L_CIRC/inference.pkl"

with open(src, "rb") as f:
    df, encoder_dict, model, posterior_samples, = pickle.load(f);

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
named_params = [site.a, site.b, site.L, site.ell, site.H]
params = [posterior_samples[param][...] for param in named_params]

for named_param, param in zip(named_params, params):
    print(named_param, param.shape)

a (4000, 8, 21, 6)
b (4000, 8, 21, 6)
L (4000, 8, 21, 6)
ℓ (4000, 8, 21, 6)
H (4000, 8, 21, 6)


In [4]:
compound_positions = encoder_dict["compound_position"].inverse_transform(sorted(set(df["compound_position"]))).tolist() #sorted(set(df["compound_position"])

vertices = [(i, cpos) for i, cpos in enumerate(compound_positions) if cpos.split("-")[0] == ""]
radii = [(i, cpos) for i, cpos in enumerate(compound_positions) if cpos.split("-")[0] and cpos.split("-")[1] == "C6LC"]
diameters = [(i, cpos) for i, cpos in enumerate(compound_positions) if (i, cpos) not in vertices and (i, cpos) not in radii]

vertices = [i for (i, cpos) in vertices]
radii = [i for (i, cpos) in radii]
diameters = [i for (i, cpos) in diameters]

In [5]:
subjects = sorted(set(df["participant"]))
positions = sorted(set(df["compound_position"]))
muscles = model.response

In [6]:
a = posterior_samples[site.a][...]

In [7]:
a.shape

(4000, 8, 21, 6)

In [8]:
encoder_dict["compound_position"].inverse_transform(vertices)

array(['-C6LC', '-C6LE', '-C6LN', '-C6LNE', '-C6LNW', '-C6LS', '-C6LSE',
       '-C6LSW', '-C6LW'], dtype=object)

In [9]:
encoder_dict["compound_position"].inverse_transform(radii)

array(['C6LE-C6LC', 'C6LN-C6LC', 'C6LNE-C6LC', 'C6LNW-C6LC', 'C6LS-C6LC',
       'C6LSE-C6LC', 'C6LSW-C6LC', 'C6LW-C6LC'], dtype=object)

In [42]:
encoder_dict["compound_position"].inverse_transform(diameters)

array(['C6LE-C6LW', 'C6LNE-C6LSW', 'C6LS-C6LN', 'C6LSE-C6LNW'],
      dtype=object)

In [43]:
norm_y = []
norm_x = []
subset = diameters
target_position = 2

for subject_ind, subject in enumerate(subjects):
    for m_ind, m in enumerate(muscles):
        # Fix the subject index and size index
        a_temp = a[:, subject_ind, subset, m_ind]
        mean_thres = np.nanmean(a_temp[:,target_position,...],axis=(0))
        curr_params = [
            params[:, subject_ind, subset, m_ind, None] for params in params
        ]

        x_temp = np.array([1.2*mean_thres])[None, None, None, :]

        temp_thresh = F.rectified_logistic(
            x_temp, #x_temp
            *curr_params
        )
        
        # Remove offset
        temp_thresh = temp_thresh - curr_params[2]
        norm_y.append(temp_thresh)
        norm_x.append(x_temp)

norm_x = np.array(norm_x)
norm_x = norm_x.reshape(len(subjects), len(muscles),*norm_x.shape[2:])
norm_y = np.array(norm_y)
norm_y = norm_y.reshape(len(subjects), len(muscles),*norm_y.shape[2:])
norm_y = ma.masked_invalid(norm_y)

In [44]:
norm_y.shape

(8, 6, 4000, 4, 1)

In [45]:
y_means = ma.mean(norm_y, axis=(2,4))

In [46]:
y_means.shape

(8, 6, 4)

In [47]:
y_diffs = y_means[:,:,:, None] - y_means[:,:,None,:]

In [48]:
y_diffs.shape

(8, 6, 4, 4)

In [49]:
y_diffs = ma.mean(y_diffs, axis=1)

In [50]:
pvalues = []
for i in range(len(diameters)):
    for i2 in range(len(diameters)):
        if i==i2:
            pvalues.append(1)
            continue
        temp_p = stats.wilcoxon(y_diffs[:, i,i2], axis=0).pvalue
        pvalues.append(temp_p)

pvalues = np.array(pvalues)
pvalues = pvalues.reshape(len(diameters), len(diameters))

In [51]:
pvalues.shape

(4, 4)

In [52]:
labels = encoder_dict['compound_position'].inverse_transform(diameters)

mask = np.tril(np.ones_like(pvalues), k=0).astype(bool)
sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask, annot=False)
sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask,annot=np.round(y_diffs.mean(axis=0).data, 3), annot_kws={'va':'top'}, fmt="", cbar=False)
sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask,annot=np.round(pvalues, 3), annot_kws={'va':'bottom'}, fmt="", cbar=False)

<Axes: >

In [30]:
def mep_graph(sub,target):
    norm_y = []
    norm_x = []
    subset = sub
    target_position = target

    for subject_ind, subject in enumerate(subjects):
        for m_ind, m in enumerate(muscles):
            # Fix the subject index and size index
            a_temp = a[:, subject_ind, subset, m_ind]
            mean_thres = np.nanmean(a_temp[:,target_position,...],axis=(0))
            curr_params = [
                params[:, subject_ind, subset, m_ind, None] for params in params
            ]

            x_temp = np.array([1.2*mean_thres])[None, None, None, :]

            temp_thresh = F.rectified_logistic(
                x_temp, #x_temp
                *curr_params
            )
            
            # Remove offset
            temp_thresh = temp_thresh - curr_params[2]
            norm_y.append(temp_thresh)
            norm_x.append(x_temp)

    norm_x = np.array(norm_x)
    norm_x = norm_x.reshape(len(subjects), len(muscles),*norm_x.shape[1:])
    norm_y = np.array(norm_y)
    norm_y = norm_y.reshape(len(subjects), len(muscles),*norm_y.shape[1:])
    norm_y = ma.masked_invalid(norm_y)

    y_means = ma.mean(norm_y, axis=(2,4))

    y_diffs = y_means[:,:, :,None] - y_means[:,:,None,:]
    y_diffs = ma.mean(y_diffs, axis=1)

    pvalues = []
    for i in range(len(subset)):
        for i2 in range(len(subset)):
            if i==i2:
                pvalues.append(1)
                continue
            temp_p = stats.wilcoxon(y_diffs[:, i,i2], axis=0).pvalue
            pvalues.append(temp_p)

    pvalues = np.array(pvalues)
    pvalues = pvalues.reshape(len(subset), len(subset))

    labels = encoder_dict['compound_position'].inverse_transform(subset)

    mask = np.tril(np.ones_like(pvalues), k=0).astype(bool)
    sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask, annot=False)
    sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask,annot=np.round(y_diffs.mean(axis=0).data, 3), annot_kws={'va':'top'}, fmt="", cbar=False)
    sns.heatmap(pvalues, xticklabels=labels, yticklabels=labels, mask = mask,annot=np.round(pvalues, 3), annot_kws={'va':'bottom'}, fmt="", cbar=False)

In [24]:
encoder_dict["compound_position"].inverse_transform(vertices)

array(['-C6LC', '-C6LE', '-C6LN', '-C6LNE', '-C6LNW', '-C6LS', '-C6LSE',
       '-C6LSW', '-C6LW'], dtype=object)

In [31]:
mep_graph(vertices,0)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (81,) + inhomogeneous part.